# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='./chromedriver')

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

In [2]:
# 模擬使用者操作行為，選擇/點擊

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()

In [4]:
# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
print(table)

# 把table資料存成array
table_g = soup.find(class_='TABLE_G')
tbody = table_g.find('tbody')
cnt = 0
data_arr = []
data = {}
all_data = {}
for i in tbody.find_all('tr', style="color:Black;"):
#     print(i, '\n######\n')
    for j in i.find_all('td'):
        print('cnt: ', cnt)
        print(j.text)
        if j.text == '\xa0':
            continue
        else:
            data_arr.append(j.text)
            
        cnt += 1
#         if (j.text) == '\xa0':
#             print('沒有東西呵呵呵！')
        if cnt > 37: 
            print('############')
            cnt = 0

# 把array資料整理成dict後用panda建立表單
ccnt26 = 0
data = {}
a = {}
for k in range(len(data_arr)):
#     print(data_arr[k])

    if k%2 == 0:
        print('k: ', k)
        print('data_arr[k]: ', data_arr[k])
        print('data_arr[k+1]: ', data_arr[k+1])
        if k == 0:
            data.setdefault(data_arr[k],{})
        else:
            if k%26 == 0:
                ccnt26 += 1
                data.setdefault(data_arr[k],{})
            else:
                print('26*ccnt26: ', 26*ccnt26)
                print('data[data_arr[26*ccnt26]]: ', data[data_arr[26*ccnt26]])
                data[data_arr[26*ccnt26]].setdefault(data_arr[k],data_arr[k+1])
#         print(data)
        
data

<table align="Center" border="1" cellpadding="3" cellspacing="0" class="TABLE_G" id="ctl05_gv" rules="all" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">
<tbody><tr style="color:Black;font-weight:normal;">
<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
</tr><tr style="color:Black;">
<td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">SO2</td><td class="no-alt" rowspan="12" valign="top">ppb</td><td>2018/01</td><td>1.80</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/02</td><td>1.90</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/03</td><td>2.20</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/04</td><td>2.30</td><td> </td>
</tr><tr style="color:Black;">
<td>2018/05</td><td>3.10</td><td> </td>
</tr><tr class="ALT" style="color:Black;">
<td>2018/06</td><td>2.70</td><td> </td>
</tr><tr style="color:Black;">
<td>

IndexError: list index out of range

In [5]:
import pandas as pd
a = pd.DataFrame(data)
a

,SO2,CO,O3,PM10,NOx,NO,NO2,THC
2018/01,1.80,0.34,33.40,23,14.17,3.41,10.77,NaN
2018/02,1.90,0.44,32.50,41,19.43,4.48,14.95,NaN
2018/03,2.20,0.40,35,39,18.50,4.93,13.56,NaN
2018/04,2.30,0.38,38.40,48,15.24,2.60,12.64,NaN
2018/05,3.10,0.34,31.60,37,15.20,2.26,12.94,NaN
2018/06,2.70,0.29,29.50,26,13.79,2.41,11.38,NaN
2018/07,2.20,0.21,18.70,24,11.60,2.68,8.92,NaN
2018/08,2.40,0.30,26.40,26,14.75,2.68,12.08,NaN
2018/09,2.10,0.26,29.10,28,12.40,2.41,10,NaN
2018/10,1.70,0.29,45.90,33,12.45,2.13,10.33,NaN


In [6]:
a.SO2[0:8]

2018/01    1.80
2018/02    1.90
2018/03    2.20
2018/04    2.30
2018/05    3.10
2018/06    2.70
2018/07    2.20
2018/08    2.40
Name: SO2, dtype: object

### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [12]:
[a.SO2, a.CO]

[2018/01    1.80
 2018/02    1.90
 2018/03    2.20
 2018/04    2.30
 2018/05    3.10
 2018/06    2.70
 2018/07    2.20
 2018/08    2.40
 2018/09    2.10
 2018/10    1.70
 2018/11    1.90
 2018/12    1.80
 無此測項        NaN
 ppm         NaN
 CH4         NaN
 Name: SO2, dtype: object,
 2018/01    0.34
 2018/02    0.44
 2018/03    0.40
 2018/04    0.38
 2018/05    0.34
 2018/06    0.29
 2018/07    0.21
 2018/08    0.30
 2018/09    0.26
 2018/10    0.29
 2018/11    0.30
 2018/12    0.35
 無此測項        NaN
 ppm         NaN
 CH4         NaN
 Name: CO, dtype: object]